In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [3]:
## Load the data set

data = pd.read_csv('Churn_Modelling.csv')
data.head()

RowNumber  CustomerId   Surname  ...  IsActiveMember EstimatedSalary Exited
0          1    15634602  Hargrave  ...               1       101348.88      1
1          2    15647311      Hill  ...               1       112542.58      0
2          3    15619304      Onio  ...               0       113931.57      1
3          4    15701354      Boni  ...               0        93826.63      0
4          5    15737888  Mitchell  ...               1        79084.10      0

[5 rows x 14 columns]

In [4]:
## Preeprocess the data
### Drop the irrelevant features/columns 
## axis=1 means column wise

data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.head()

CreditScore Geography  Gender  ...  IsActiveMember  EstimatedSalary  Exited
0          619    France  Female  ...               1        101348.88       1
1          608     Spain  Female  ...               1        112542.58       0
2          502    France  Female  ...               0        113931.57       1
3          699    France  Female  ...               0         93826.63       0
4          850     Spain  Female  ...               1         79084.10       0

[5 rows x 11 columns]

In [54]:
## Encode categorical variable 
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data


CreditScore  Gender  ...  Geography_Germany  Geography_Spain
0             619       0  ...                0.0              0.0
1             608       0  ...                0.0              1.0
2             502       0  ...                0.0              0.0
3             699       0  ...                0.0              0.0
4             850       0  ...                0.0              1.0
...           ...     ...  ...                ...              ...
9995          771       1  ...                0.0              0.0
9996          516       1  ...                0.0              0.0
9997          709       0  ...                0.0              0.0
9998          772       1  ...                1.0              0.0
9999          792       0  ...                0.0              0.0

[10000 rows x 13 columns]

In [9]:
## One hot encode 'Geography' column

from sklearn.preprocessing import OneHotEncoder
one_hot_encoder_geo = OneHotEncoder()

geo_encoder = one_hot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [10]:
one_hot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [11]:
geo_encoded_df= pd.DataFrame(geo_encoder.toarray(),columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

Geography_France  Geography_Germany  Geography_Spain
0                  1.0                0.0              0.0
1                  0.0                0.0              1.0
2                  1.0                0.0              0.0
3                  1.0                0.0              0.0
4                  0.0                0.0              1.0
...                ...                ...              ...
9995               1.0                0.0              0.0
9996               1.0                0.0              0.0
9997               1.0                0.0              0.0
9998               0.0                1.0              0.0
9999               1.0                0.0              0.0

[10000 rows x 3 columns]

In [ ]:
## Combine one hot encoder columns with the orignal data
data = pd.concat([data.drop('Geography',axis=1), geo_encoded_df],axis=1)


In [14]:
data

CreditScore  Gender  ...  Geography_Germany  Geography_Spain
0             619       0  ...                0.0              0.0
1             608       0  ...                0.0              1.0
2             502       0  ...                0.0              0.0
3             699       0  ...                0.0              0.0
4             850       0  ...                0.0              1.0
...           ...     ...  ...                ...              ...
9995          771       1  ...                0.0              0.0
9996          516       1  ...                0.0              0.0
9997          709       0  ...                0.0              0.0
9998          772       1  ...                1.0              0.0
9999          792       0  ...                0.0              0.0

[10000 rows x 13 columns]

In [53]:
## Save the Encoder and scaler
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)



with open('one_hot_encoder_geo.pkl','wb') as file:
    pickle.dump(one_hot_encoder_geo,file)

In [17]:
## Devide dataset into dependent and Independent features
x = data.drop('Exited',axis=1)
y = data['Exited']


## Split the data in training and testing sets.
X_train, X_test, Y_train,Y_test =train_test_split(x,y,test_size=0.2,random_state=42)

##Scale these features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [18]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]], shape=(8000, 12))

In [19]:
## Save scalar file to pickle
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

## ANN Impementation


In [31]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [27]:
(X_train.shape[1],)

(12,)

In [28]:
## Build our ANN model
model = Sequential([
    Dense(64,activation='relu',input_shape =(X_train.shape[1],)), ## Hidden Layer1 connected with input layer
    Dense(32,activation='relu'), ## Hidden Layer2 connected with Hidden Layer1
    Dense(1,activation='sigmoid') ## Output layer
]
)

d:\Ai_Practice\Python\venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [29]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [32]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy()

In [33]:
## Compile the model.
model.compile(optimizer=opt,loss=loss,metrics=['accuracy'])


In [35]:
## Setup the Tensorboard
log_dir = "logs/fit/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir,histogram_freq=1)

In [37]:
##Setup Early stopping 
early_stopping_callback =  EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [39]:
## Train the model
history = model.fit(
    X_train,Y_train,validation_data =(X_test,Y_test),epochs =100,
    callbacks = [tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8625 - loss: 0.3378 - val_accuracy: 0.8565 - val_loss: 0.3458
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8634 - loss: 0.3356 - val_accuracy: 0.8510 - val_loss: 0.3613
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8646 - loss: 0.3327 - val_accuracy: 0.8590 - val_loss: 0.3490
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8640 - loss: 0.3350 - val_accuracy: 0.8590 - val_loss: 0.3455
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8650 - loss: 0.3305 - val_accuracy: 0.8560 - val_loss: 0.3519
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8656 - loss: 0.3265 - val_accuracy: 0.8620 - val_loss: 0.3478
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8636 - loss: 0.3254 - val_accuracy: 0.8575 - val_loss: 0.3469
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8712 - loss: 0.3206 - val_accu

In [ ]:
##Save the model to file
model.save('model.h5')

In [43]:
## Load Tensorboard Extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [46]:
%tensorboard --logdir logs/fit/20250811-210615

Reusing TensorBoard on port 6007 (pid 44560), started 0:00:49 ago. (Use '!kill 44560' to kill it.)